# Realizzazione di un sistema ibrido

Il sistema sarà in grado di riconoscere la tipologia di domanda (globale/locale) ed utilizzare la pipeline più adeguata a quella tipologia.

In [117]:
import pandas as pd
import numpy as np
import json
import requests

## Classificatore di domande

La classificazione viene effettuata tramite una chiamata ad un LLM con apposito prompt, che fornisce descrizioni ed esempi delle due tipologie di domande.

### Prompt per la classificazione

In [118]:
classification_prompt = """
Task: You will classify the following question as either "local" or "global". Please respond only with the word "local" or "global".

Context:
We are analyzing a set of 9 papers related to time-series models. The titles of the 9 papers are:

1. TranAD
2. AnomalyBERT
3. RestAD
4. TimeGPT
5. TimeLLM
6. LagLlama
7. Chronos
8. Foundation Models for Time Series Analysis
9. TimeSFM

Definitions:
- **Local questions**: These questions focus on specific details found within a single paper. These questions often reference sections, figures, or tables of a document. They do not require synthesizing information from multiple papers on the list, even if they involve comparisons between different methods within the same paper.
 Characteristics of Local Questions:
  1. They reference specific sections, tables, or figures within a single document.
  2. Formed sometimes by very specific questions, dealing with very particular technical details.
  3. If it says 'according to the abstract of the paper... ', 'in accordance with table ..', 'according to section ..', the specific chapter, or uses phrases like that, the question is almost certainly local.
  4. They do not require synthesizing information from multiple papers on the list, even if they involve comparisons between different methods within the same paper.
  5. Sometimes it focuses on detailed results, scores, metrics, precise comparisons with numerical results.
 Examples of local questions:
    1. "What is the architecture of the model described in Section 4 of the paper?"
    2. "How does the model's performance in Table 2 compare to other baselines presented in the same paper?"
    3. "What preprocessing steps are necessary for the model described in this paper to handle seasonal data?"
    4. "What evaluation metrics were used in the analysis presented in Section 5 of the paper?"

    
- **Global questions**: These questions require the synthesis of information. They require understanding an entire paper and synthesizing it, or they require comparisons between the papers mentioned above. Global questions are also those that address general themes common to all papers.
 Characteristics of Global Questions:
  1. They compare or reference multiple papers or models from the list of 9 papers.
  2. They often ask about the trade-offs, similarities, or differences between methods from different documents on the list.
  3. They require a broader understanding or a synthesis of information across papers or models from the list.
  4. They can also be just about a specific paper but require a summary or the main contents of the paper 
 Examples of Global Questions (synthetic examples):
    1. "How does the model Y compare to model Z for anomaly detection?"
    2. "What are the key differences between the forecasting models described in Papers C and D?"
    3. "How do the models from Paper F and Paper I handle non-stationary data, and why is this important?"
    4. "How does Model X work?"

Additional Instructions:
 - If different papers, found in the list posed above, are mentioned in the question, the question will definitely be “global”.
 - If the question includes a comparison but one of the models mentioned is not in the list of papers, then it refers to a comparison within a paper and the question is “local”.
 - If the question mentions a specific section of a paper, such as the abstract, chapter number, table, code, image, or other, the question will definitely be “local”.
Question:
{query}

Answer: 
"""

### Chiamata al LLM

Vengono definite le funzioni per la chiamata al LLM per la classificazione della domanda.

Vari prompt sono stati testati a parte per valutarne le performance di classificazione.

Le funzioni verranno poi usate nella pipeline definitiva.

In [119]:
model_url = 'http://172.18.21.132:8000/v1/completions'
model_name = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

def generate_payload(question, temperature=0.0, max_tokens=1):
    prompt = classification_prompt.format(query=question)
    
    payload = {
        "model": model_name,
        "prompt": prompt,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    return payload

In [120]:
def get_classification(question):
    payload = generate_payload(question)
    headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(model_url, headers=headers, json=payload)
    
    if response.status_code == 200:
        return response.json()["choices"][0]["text"].strip()
    else:
        print(f"Errore nella richiesta: {response.status_code}")
        print(response.text)
        return None

## Definizione delle due pipeline

### GraphRAG per pipeline globale

In [121]:
import tiktoken
import os
from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch
import asyncio

In [122]:
file_path = '../../output/20240925-154939/artifacts'

if not os.path.exists(file_path) or not os.listdir(file_path):
    print("The specified path is empty or does not exist.")
else:
    print("The path exists and is not empty.")

The path exists and is not empty.


In [123]:
INPUT_DIR = file_path
LANCEDB_URI = f"{INPUT_DIR}/lancedb"

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"
RELATIONSHIP_TABLE = "create_final_relationships"
COVARIATE_TABLE = "create_final_covariates"
TEXT_UNIT_TABLE = "create_final_text_units"

COMMUNITY_LEVEL = 4

In [124]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
nodes_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")
relationship_df = pd.read_parquet(f"{INPUT_DIR}/{RELATIONSHIP_TABLE}.parquet")
df_report = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")

In [125]:
token_encoder = tiktoken.get_encoding("cl100k_base")

In [126]:
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)

entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)

In [127]:
from dotenv import load_dotenv

load_dotenv()

True

In [128]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
api_base = "http://172.18.21.132:8000/v1"

In [129]:
llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  
    api_base=api_base,  
    max_retries=20,
)

In [130]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  
    token_encoder=token_encoder,
)

In [131]:
context_builder_params = {
    "use_community_summary": True,  
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 6000,  
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1500,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 1500,  
    "temperature": 0.0,
}

In [132]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=6000, 
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  
    json_mode=True,  
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="Single page",  
)

In [133]:
async def ask_question(question):
    try:
        result = await search_engine.asearch(question)  
        answer = result.response
    except Exception as e:
        print(f"Error processing question: {question}\nException: {e}")
        answer = "Error: Unable to retrieve answer."
    return answer

In [134]:
async def global_pipeline(question):
    print(f"Running global pipeline for question: {question}")
    answer = await ask_question(question) 
    return answer

### RAG tradizionale per pipeline locale

In [135]:
from langchain.embeddings.base import Embeddings
from langchain.llms.base import LLM
from typing import Optional, List
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import glob
from langchain.docstore.document import Document

In [136]:
class CustomEmbeddings(Embeddings):
    def __init__(self, endpoint_url):
        self.endpoint_url = endpoint_url

    def embed_documents(self, texts):
        embeddings = []
        for text in texts:
            payload = {
                "input": text,
                "model": "intfloat/multilingual-e5-large-instruct"
            }
            response = requests.post(f"{self.endpoint_url}/embeddings", json=payload)
            if response.status_code == 200:
                embedding = response.json()['data'][0]['embedding']
                embeddings.append(embedding)
            else:
                raise Exception(f"Errore nell'embedder: {response.text}")
        return embeddings

    def embed_query(self, text):
        return self.embed_documents([text])[0]

embedder = CustomEmbeddings(endpoint_url="http://172.18.21.138:80/v1")

In [137]:
document_paths = glob.glob('../input/*.txt')

In [138]:
documents = []
for file_path in document_paths:
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
        documents.append(Document(page_content=content))

In [139]:
print(f"Numero di documenti caricati: {len(documents)}")

Numero di documenti caricati: 9


In [140]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,       
    chunk_overlap=200,     
    separators=["\n\n", "\n", ".", "!", "?", ",", " ", ""],
    length_function=len
)

docs = text_splitter.split_documents(documents)

print(f"Numero totale di chunk: {len(docs)}\n")

Numero totale di chunk: 718



In [141]:
vectorstore = FAISS.from_documents(docs, embedder)

In [142]:
prompt_template = """
You are a knowledgeable assistant specialized in answering questions based solely on the provided context. Provide a detailed and well-structured answer, including all relevant information from the context. Ensure your response is comprehensive, faithful to the context, and presented in clear, well-formed sentences. Do not add any information that is not present in the context. If the answer is not explicitly stated in the context, respond with "I don't know."

Context:
{context}

Question:
{question}

Answer:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [143]:
class CustomLLM(LLM):
    endpoint_url: str
    model_name: str = "meta-llama/Meta-Llama-3.1-8B-Instruct"
    temperature: float = 0.0
    max_tokens: int = 1500
    repetition_penalty: float = 1.2

    @property
    def _llm_type(self) -> str:
        return "custom_llm"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        payload = {
            "prompt": prompt,
            "model": self.model_name,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "repetition_penalty": self.repetition_penalty,
            "stop": stop or ["I don't know."],
        }
        print("Payload inviato all'API:", payload)  
        response = requests.post(f"{self.endpoint_url}/completions", json=payload)
        if response.status_code == 200:
            return response.json()['choices'][0]['text']
        else:
            raise Exception(f"Errore nel LLM: {response.text}")

llm = CustomLLM(
    endpoint_url="http://172.18.21.132:8000/v1",
    temperature=0.0,
    max_tokens= 500
)

In [144]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

In [145]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=False,
    chain_type_kwargs={"prompt": prompt}
)

In [146]:
def ask_question_naive(query):  
    result = qa_chain({"query": query})
    return result['result']

In [147]:
def local_pipeline(question):
    print(f"Running local pipeline (naive RAG) for question: {question}")
    answer = ask_question_naive(question)  
    return answer

## Pipeline finale

In [148]:
async def process_question(question):
    classification = get_classification(question)  
    if classification == "local":
        result = local_pipeline(question)  
    elif classification == "global":
        result = await global_pipeline(question)  
    else:
        result = "Error: Invalid classification"
    
    return result

In [149]:
async def process_all_questions(questions):
    tasks = [process_question(question) for question in questions]
    answers = await asyncio.gather(*tasks)  
    return answers

In [150]:
async def main():
    answers = await process_all_questions(questions_list)
    
    model_answers = []
    for question, answer in zip(questions_list, answers):
        model_answers.append({
            "question": question,
            "answer": answer
        })
    

    output_file = 'questions_answers.json'
    with open(output_file, 'w') as f:
        json.dump(model_answers, f, indent=4)
    
    print(f"Results saved in {output_file}")

## Estrazione di tutte le domande

In [151]:
questions_path = '../DatasetCreation/Labeled_data.json'

In [152]:
labeled_data = pd.read_json(questions_path)
print(labeled_data.head())
print(labeled_data.columns)

                                                   0       1
0  What are the main topics covered by the data i...  global
1  How does RestAD leverage both statistical meth...  global
2  What are the key features and benefits of Rest...  global
3  What are the key features and benefits of Rest...  global
4  How does TimeLLM differ from other models in t...  global
RangeIndex(start=0, stop=2, step=1)


In [153]:
questions_list = labeled_data[0].values
print(questions_list)

['What are the main topics covered by the data in the set of time-series papers?'
 'How does RestAD leverage both statistical methods and machine learning to achieve robust anomaly detection in noisy time-series data?'
 'What are the key features and benefits of RestAD in anomaly detection for time-series data?'
 'What are the key features and benefits of RestAD in anomaly detection for time-series data?'
 'How does TimeLLM differ from other models in time-series forecasting?'
 'How does AnomalyBERT work?'
 'How does TimeGPT approach time-series forecasting?'
 'What types of real-world applications can benefit from models like TimeLLM, RestAD, TimeGPT, AnomalyBERT, LagLLama and the other models described?'
 'What distinguishes LagLLama in its approach to time-series analysis?'
 'How do models like AnomalyBERT handle non-stationary data, and why is this important?'
 'What are the main trade-offs when choosing between transformer-based models and traditional time-series models?'
 'How do

## Esecuzione della pipeline

In [154]:
try:
    loop = asyncio.get_event_loop()
    if loop.is_running():
        print("Using existing event loop...")
        await main()  
    else:
        print("Creating a new event loop...")
        loop.run_until_complete(main())
except RuntimeError:
    print("Creating a new event loop...")
    asyncio.run(main())

Using existing event loop...
Running global pipeline for question: What are the main topics covered by the data in the set of time-series papers?
Running global pipeline for question: How does RestAD leverage both statistical methods and machine learning to achieve robust anomaly detection in noisy time-series data?
Running global pipeline for question: What are the key features and benefits of RestAD in anomaly detection for time-series data?
Running global pipeline for question: What are the key features and benefits of RestAD in anomaly detection for time-series data?
Running global pipeline for question: How does TimeLLM differ from other models in time-series forecasting?
Running local pipeline (naive RAG) for question: How does AnomalyBERT work?
Payload inviato all'API: {'prompt': '\nYou are a knowledgeable assistant specialized in answering questions based solely on the provided context. Provide a detailed and well-structured answer, including all relevant information from the c